In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn import metrics
import pickle

In [ ]:
#Seleccionar ruta de la data
df = pd.read_csv('../data/modeling/02_preprocessed.csv', sep="|", encoding='latin')
df.shape

In [ ]:
cols = ['RATIO_NO_TOTAL_SF_HAB','VARIACIO_1M_TOTAL_DEUDA']

In [ ]:
corr_df = df[cols].corr(method='pearson')
print(corr_df)

In [ ]:
#entrenamiento del modelo
model = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.002), random_state=7)#cambiar la contaminacion para competir modelos.
model.fit(df[cols])

In [ ]:
#PREDICCION
df['anomalies_scores'] = model.decision_function(df[cols])
df['anomaly'] = model.predict(df[cols])
df.head(20)

In [ ]:
scoreSilhoutte = metrics.silhouette_score(df[cols], df['anomaly'], metric='euclidean', sample_size = 10000, random_state=666)
print(scoreSilhoutte)

In [ ]:
df.groupby(['CODMES','anomaly']).size()

In [ ]:
df_puno = df.loc[(df['CODCLAVECIC']==11258692)]
df_puno[['CODMES','CODCLAVECIC', 'RATIO_NO_TOTAL_SF_HAB','VARIACIO_1M_TOTAL_DEUDA', 'anomaly']].head(30)

In [ ]:
df_outliers = df.loc[(df['anomaly']==-1)]
df_outliers.head(20)


In [ ]:
# save the model to disk
pickle.dump(model, open('../src/02_models/if_model_39.model', 'wb'))

In [ ]:
df.to_csv('../data/modeling/04_predicted.csv',index=False,sep='|')